In [1]:
model_name = '7b'
output_dir = 'OUTPUTS'
model_dir = 'models'
exp_name = 'bulario_questions'

In [2]:
import bullama.util
from bullama.util import create_model_tokenizer

model, tokenizer = create_model_tokenizer(model_name, True, False)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/navarro/miniconda3/envs/py39/lib/python3.9/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [3]:
import dataset_prompt

dataset = dataset_prompt.generate_dataset("datasets/bulario_prompts_treino.csv", split = False)

In [4]:
dataset

Dataset({
    features: ['text'],
    num_rows: 7226
})

In [5]:
dataset[0]

{'text': '[INST] <<SYS>> Responda apenas o que for pedido, usando o contexto:<</SYS>>    \n    Context: Abilify comprimidos de 5, 10, 15, 20 ou 30 mg é indicado no tratamento de esquizofrenia em adultos e adolescentes com mais de 15 anos.\u200b  Além disso, estes comprimidos de ingestão oral, também podem ser utilizados no tratamento e prevenção de episódios maníacos na perturbação bipolar 1 em adultos e adolescentes com mais de 13 anos.  Abilify comprimidos também pode ser encontrado na forma de comprimidos orodispersíveis que dissolvem na boca sem uso de água, sendo indicados para pacientes idosos, agitados ou com problemas para engolir.   As doses de Abilify variam de acordo com a idade do paciente e a doença a tratar, sendo que as orientações gerais incluem:  Adultos  Esquizofrenia: a dose inicial é de 10 a 15 mg uma vez por dia, independente das refeições. Já a dose de manutenção, normalmente, é de 15 mg por dia, podendo ser aumentada até 30 mg de acordo com indicação do médico.\t

In [6]:
model = bullama.util.prepare_model(model)

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
training_args = TrainingArguments(
    per_device_train_batch_size = 1, #The batch size per GPU (default: 8)
    gradient_accumulation_steps = 1,
    warmup_steps = 0, # Number of steps used for a linear warmup from 0 to learning_rate (default: 0)
    num_train_epochs=3,
   # weight_decay=0.1,
    learning_rate = 2e-4,
    fp16 = True, # Whether to use fp16 16-bit (mixed) precision training instead of 32-bit training.
    logging_steps = 1, # Number of update steps between two logs if logging_strategy="steps". Should be an integer or a float in range [0,1). 
    #If smaller than 1, will be interpreted as ratio of total training steps.
    overwrite_output_dir = True,
   # evaluation_strategy = "epoch", #"The evaluation strategy to adopt during training. "
  # save_strategy = "no",
    push_to_hub = False,
    output_dir = output_dir,
    report_to="tensorboard",
    optim="paged_adamw_8bit",
)

In [8]:
from trl import SFTTrainer
trainer_sft =  SFTTrainer(
    model = model,
    train_dataset=dataset,
#    eval_dataset=dataset['valid'],
    peft_config = bullama.util.get_lora_config(),
    dataset_text_field = "text",
    max_seq_length = 2048,
    tokenizer=tokenizer,
    args=training_args
)

Map:   0%|          | 0/7226 [00:00<?, ? examples/s]

In [9]:
trainer_sft.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/navarro/miniconda3/envs/py39/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/navarro/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,2.204600
2,1.876000
3,1.600400
4,1.869700
5,1.403900
6,1.582000
7,1.364300
8,1.164900
9,1.768600
10,1.406200


/home/navarro/miniconda3/envs/py39/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/navarro/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/navarro/miniconda3/envs/py39/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of 

TrainOutput(global_step=21678, training_loss=0.3217825408773972, metrics={'train_runtime': 27914.3423, 'train_samples_per_second': 0.777, 'train_steps_per_second': 0.777, 'total_flos': 1.116520751619883e+18, 'train_loss': 0.3217825408773972, 'epoch': 3.0})

In [11]:
trainer_sft.save_model(f"{model_dir}/{exp_name}")
tokenizer.save_pretrained(f"{model_dir}/{exp_name}")

('models/bulario_questions/tokenizer_config.json',
 'models/bulario_questions/special_tokens_map.json',
 'models/bulario_questions/tokenizer.json')